In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nri-by-county/National_Risk_Index_Counties_807384124455672111.csv


In [3]:
# Show all columns without truncation
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
import pandas as pd
import json
import re



In [4]:
# 1) Path to original FEMA NRI Counties CSV 
IN_PATH  = "/kaggle/input/nri-by-county/National_Risk_Index_Counties_807384124455672111.csv"

# 2) Output path for cleaned file (ready to import to Supabase)
OUT_PATH = "/kaggle/working/nri_counties_clean.csv"

In [5]:
# Load
df = pd.read_csv(IN_PATH)

In [6]:
df.shape

(3231, 468)

In [7]:
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,OBJECTID,National Risk Index ID,State Name,State Name Abbreviation,State FIPS Code,County Name,County Type,County FIPS Code,State-County FIPS Code,Population (2020),Building Value ($),Agriculture Value ($),Area (sq mi),National Risk Index - Value - Composite,National Risk Index - Score - Composite,National Risk Index - Rating - Composite,National Risk Index - State Percentile - Composite,Expected Annual Loss - Score - Composite,Expected Annual Loss - Rating - Composite,Expected Annual Loss - State Percentile - Composite,Expected Annual Loss - Total - Composite,Expected Annual Loss - Building Value - Composite,Expected Annual Loss - Population - Composite,Expected Annual Loss - Population Equivalence - Composite,Expected Annual Loss - Agriculture Value - Composite,Expected Annual Loss Rate - Building - Composite,Expected Annual Loss Rate - Population - Composite,Expected Annual Loss Rate - Agriculture - Composite,Expected Annual Loss Rate - National Percentile - Composite,Social Vulnerability and Community Resilience Adjusted Expected Annual Loss Rate - National Percentile - Composite,Social Vulnerability - Score,Social Vulnerability - Rating,Social Vulnerability - State Percentile,Community Resilience - Score,Community Resilience - Rating,Community Resilience - State Percentile,Community Resilience - Value,Community Risk Factor - Value,Avalanche - Number of Events,Avalanche - Annualized Frequency,Avalanche - Exposure - Impacted Area (sq mi),Avalanche - Exposure - Building Value,Avalanche - Exposure - Population,Avalanche - Exposure - Population Equivalence,Avalanche - Exposure - Total,Avalanche - Historic Loss Ratio - Buildings,Avalanche - Historic Loss Ratio - Population,Avalanche - Historic Loss Ratio - Total Rating,Avalanche - Expected Annual Loss - Building Value,Avalanche - Expected Annual Loss - Population,Avalanche - Expected Annual Loss - Population Equivalence,Avalanche - Expected Annual Loss - Total,Avalanche - Expected Annual Loss Score,Avalanche - Expected Annual Loss Rating,Avalanche - Expected Annual Loss Rate - Building,Avalanche - Expected Annual Loss Rate - Population,Avalanche - Expected Annual Loss Rate - National Percentile,Avalanche - Hazard Type Risk Index Value,Avalanche - Hazard Type Risk Index Score,Avalanche - Hazard Type Risk Index Rating,Coastal Flooding - Number of Events,Coastal Flooding - Annualized Frequency,Coastal Flooding - Exposure - Impacted Area (sq mi),Coastal Flooding - Exposure - Building Value,Coastal Flooding - Exposure - Population,Coastal Flooding - Exposure - Population Equivalence,Coastal Flooding - Exposure - Total,Coastal Flooding - Historic Loss Ratio - Buildings,Coastal Flooding - Historic Loss Ratio - Population,Coastal Flooding - Historic Loss Ratio - Total Rating,Coastal Flooding - Expected Annual Loss - Building Value,Coastal Flooding - Expected Annual Loss - Population,Coastal Flooding - Expected Annual Loss - Population Equivalence,Coastal Flooding - Expected Annual Loss - Total,Coastal Flooding - Expected Annual Loss Score,Coastal Flooding - Expected Annual Loss Rating,Coastal Flooding - Expected Annual Loss Rate - Building,Coastal Flooding - Expected Annual Loss Rate - Population,Coastal Flooding - Expected Annual Loss Rate - National Percentile,Coastal Flooding - Hazard Type Risk Index Value,Coastal Flooding - Hazard Type Risk Index Score,Coastal Flooding - Hazard Type Risk Index Rating,Cold Wave - Number of Events,Cold Wave - Annualized Frequency,Cold Wave - Exposure - Impacted Area (sq mi),Cold Wave - Exposure - Building Value,Cold Wave - Exposure - Population,Cold Wave - Exposure - Population Equivalence,Cold Wave - Exposure - Agriculture Value,Cold Wave - Exposure - Total,Cold Wave - Historic Loss Ratio - Buildings,Cold Wave - Historic Loss Ratio - Population,Cold Wave - Historic Loss Ratio - Agriculture,Cold Wave - Historic Loss Ratio - Total Rating,Cold Wave - Expected Annual Loss - Building Value,Cold Wave - Expected Annual Loss - Population,Cold Wave - Expect

In [8]:
for col in df.columns:
    print(col)

OBJECTID
National Risk Index ID
State Name
State Name Abbreviation
State FIPS Code
County Name
County Type
County FIPS Code
State-County FIPS Code
Population (2020)
Building Value ($)
Agriculture Value ($)
Area (sq mi)
National Risk Index - Value - Composite
National Risk Index - Score - Composite
National Risk Index - Rating - Composite
National Risk Index - State Percentile - Composite
Expected Annual Loss - Score - Composite
Expected Annual Loss - Rating - Composite
Expected Annual Loss - State Percentile - Composite
Expected Annual Loss - Total - Composite
Expected Annual Loss - Building Value - Composite
Expected Annual Loss - Population - Composite
Expected Annual Loss - Population Equivalence - Composite
Expected Annual Loss - Agriculture Value - Composite
Expected Annual Loss Rate - Building - Composite
Expected Annual Loss Rate - Population - Composite
Expected Annual Loss Rate - Agriculture - Composite
Expected Annual Loss Rate - National Percentile - Composite
Social Vulnera

In [9]:
HAZARDS = [
    "Avalanche","Coastal Flooding","Cold Wave","Drought","Earthquake","Hail",
    "Heat Wave","Hurricane","Ice Storm","Landslide","Lightning","Riverine Flooding",
    "Strong Wind","Tornado","Tsunami","Volcanic Activity","Wildfire","Winter Weather"
]

COL = {
    "state_name"     : "State Name",
    "state_fips"     : "State FIPS Code",          # numeric, 1..72, etc. -> needs zfill(2)
    "county_name"    : "County Name",
    "county_code3"   : "County FIPS Code",         # numeric 1..999 -> needs zfill(3)
    "state_county_5" : "State-County FIPS Code",   # often 4 digits in your sample -> must zfill(5)
    "risk_score"     : "National Risk Index - Score - Composite",
    "risk_rating"    : "National Risk Index - Rating - Composite",
}

In [10]:
# ---- Sanity check: required columns exist
missing = [v for v in COL.values() if v not in df.columns]
if missing:
    raise ValueError(f"Input CSV missing required columns: {missing}")

In [11]:
# Drop anything that matches these patterns (case-insensitive by using (?i) flag)
regex_to_drop = (
    r"(?i)(Exposure|Historic Loss Ratio|Expected Annual Loss( Rate)?|Annualized Frequency|Value|Number of Events|"
    r"Shape__Area|Shape__Length|GlobalID|National Risk Index Version|State Name Abbreviation|"
    r"National Percentile|State Percentile|Community Risk Factor - Value)"
)
# Select columns to drop
cols_regex = df.filter(regex=regex_to_drop, axis=1).columns

# Also drop some generic IDs we don’t need
drop_exact = [
    "OBJECTID",
    "National Risk Index ID",
    "National Risk Index - Value - Composite",  # keep 'Score' & 'Rating' only
    "Population (2020)",
    "Building Value ($)",
    "Agriculture Value ($)",
    "Area (sq mi)"
]

# Union both sets
to_drop = set(cols_regex).union([c for c in drop_exact if c in df.columns])

# Drop safely
df_new = df.drop(columns=list(to_drop), errors="ignore")

print("Original cols:", len(df.columns))
print("Dropped cols :", len(to_drop))
print("Remaining cols:", len(df_new.columns))


Original cols: 468
Dropped cols : 420
Remaining cols: 48


In [12]:
# Prefer State-County FIPS Code if present; else build from state_fips + county_code3
def only_digits(s):
    if pd.isna(s):
        return None
    m = re.search(r"\d+", str(s))
    return m.group(0) if m else None

has_sc5 = COL["state_county_5"] in df_new.columns
if has_sc5:
    sc5 = df_new[COL["state_county_5"]].apply(only_digits).fillna("").str.zfill(5)
else:
    sc5 = pd.Series([""] * len(df_new))

# Fallback path: concat 2-digit state + 3-digit county (both zero-padded)
st2 = df_new[COL["state_fips"]].apply(only_digits).fillna("").str.zfill(2)
co3 = df_new[COL["county_code3"]].apply(only_digits).fillna("").str.zfill(3)
fallback_5 = st2 + co3

# Use sc5 if it looks valid (5 digits), else fallback
county_fips = sc5.where(sc5.str.fullmatch(r"\d{5}") == True, fallback_5)

# Also normalize state_fips to 2 digits
state_fips_norm = st2

In [13]:
df_new.shape

(3231, 48)

In [14]:
for col in df_new.columns:
    print(col)

State Name
State FIPS Code
County Name
County Type
County FIPS Code
State-County FIPS Code
National Risk Index - Score - Composite
National Risk Index - Rating - Composite
Social Vulnerability - Score
Social Vulnerability - Rating
Community Resilience - Score
Community Resilience - Rating
Avalanche - Hazard Type Risk Index Score
Avalanche - Hazard Type Risk Index Rating
Coastal Flooding - Hazard Type Risk Index Score
Coastal Flooding - Hazard Type Risk Index Rating
Cold Wave - Hazard Type Risk Index Score
Cold Wave - Hazard Type Risk Index Rating
Drought - Hazard Type Risk Index Score
Drought - Hazard Type Risk Index Rating
Earthquake - Hazard Type Risk Index Score
Earthquake - Hazard Type Risk Index Rating
Hail - Hazard Type Risk Index Score
Hail - Hazard Type Risk Index Rating
Heat Wave - Hazard Type Risk Index Score
Heat Wave - Hazard Type Risk Index Rating
Hurricane - Hazard Type Risk Index Score
Hurricane - Hazard Type Risk Index Rating
Ice Storm - Hazard Type Risk Index Score
Ice

In [15]:
df_new.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,State Name,State FIPS Code,County Name,County Type,County FIPS Code,State-County FIPS Code,National Risk Index - Score - Composite,National Risk Index - Rating - Composite,Social Vulnerability - Score,Social Vulnerability - Rating,Community Resilience - Score,Community Resilience - Rating,Avalanche - Hazard Type Risk Index Score,Avalanche - Hazard Type Risk Index Rating,Coastal Flooding - Hazard Type Risk Index Score,Coastal Flooding - Hazard Type Risk Index Rating,Cold Wave - Hazard Type Risk Index Score,Cold Wave - Hazard Type Risk Index Rating,Drought - Hazard Type Risk Index Score,Drought - Hazard Type Risk Index Rating,Earthquake - Hazard Type Risk Index Score,Earthquake - Hazard Type Risk Index Rating,Hail - Hazard Type Risk Index Score,Hail - Hazard Type Risk Index Rating,Heat Wave - Hazard Type Risk Index Score,Heat Wave - Hazard Type Risk Index Rating,Hurricane - Hazard Type Risk Index Score,Hurricane - Hazard Type Risk Index Rating,Ice Storm - Hazard Type Risk Index Score,Ice Storm - Hazard Type Risk Index Rating,Landslide - Hazard Type Risk Index Score,Landslide - Hazard Type Risk Index Rating,Lightning - Hazard Type Risk Index Score,Lightning - Hazard Type Risk Index Rating,Riverine Flooding - Hazard Type Risk Index Score,Riverine Flooding - Hazard Type Risk Index Rating,Strong Wind - Hazard Type Risk Index Score,Strong Wind - Hazard Type Risk Index Rating,Tornado - Hazard Type Risk Index Score,Tornado - Hazard Type Risk Index Rating,Tsunami - Hazard Type Risk Index Score,Tsunami - Hazard Type Risk Index Rating,Volcanic Activity - Hazard Type Risk Index Score,Volcanic Activity - Hazard Type Risk Index Rating,Wildfire - Hazard Type Risk Index Score,Wildfire - Hazard Type Risk Index Rating,Winter Weather - Hazard Type Risk Index Score,Winter Weather - Hazard Type Risk Index Rating
0,Alabama,1,Autauga,County,1,1001,49.220490,Relatively Low,51.299999,Relatively Moderate,51.810001,Relatively Moderate,NaN,Not Applicable,NaN,Not Applicable,0.0,No Rating,50.652243,Relatively Low,57.079224,Very Low,57.111040,Relatively Low,87.082405,Relatively Moderate,67.653656,Relatively Low,23.850766,Very Low,41.760360,Relatively Low,54.247104,Relatively Low,52.243080,Relatively Low,43.970729,Relatively Low,73.846643,Relatively Moderate,NaN,Not Applicable,NaN,Not Applicable,29.621381,Very Low,12.217626,Very Low
1,Alabama,1,Baldwin,County,3,1003,97.709195,Relatively High,31.030001,Relatively Low,86.120003,Very High,NaN,Not Applicable,55.935614,Relatively Low,0.0,No Rating,86.796055,Relatively Moderate,69.169583,Relatively Low,52.943048,Relatively Low,91.091314,Relatively Moderate,98.878421,Relatively High,32.644903,Relatively Low,72.502409,Relatively Low,99.388674,Very High,87.941457,Relatively Moderate,53.770283,Relatively Low,91.377665,Relatively Moderate,NaN,Insufficient Data,NaN,Not Applicable,92.650334,Relatively Moderate,52.083996,Relatively Low
2,Alabama,1,Barbour,County,5,1005,56.188355,Relatively Low,99.269997,Very High,6.240000,Very Low,NaN,Not Applicable,NaN,Not Applicable,0.0,No Rating,80.687241,Relatively Low,44.002545,Very Low,39.993637,Very Low,69.710468,Relatively Low,77.164648,Relatively Low,18.887408,Very Low,70.799872,Relatively Low,82.303732,Relatively Moderate,13.967547,Very Low,35.507477,Relatively Low,73.942094,Relatively Moderate,NaN,Not Applicable,NaN,Not Applicable,27.266942,Very Low,19.535476,Very Low
3,Alabama,1,Bibb,County,7,1007,32.484887,Very Low,80.779999,Very High,19.730000,Very Low,NaN,Not Applicable,NaN,Not Applicable,0.0,No Rating,32.421254,Very Low,61.692650,Very Low,43.684378,Very Low,67.228762,Relatively Low,51.099148,Very Low,43.637575,Relatively Low,41.856730,Relatively Low,39.832690,Relatively Low,25.167038,Very Low,23.258034,Relatively Low,71.969456,Relatively Moderate,NaN,Not Applicable,NaN,Not Applicable,15.049316,Very Low,11.104041,Very Low
4,Alabama,1,Blount,County,9,1009,65.128858,Relatively Low,51.369999,Relatively Moderate,22.820000,Relatively Low,NaN,Not Applicable,NaN,Not Applicable,0.0

In [16]:
# Safety check: make sure required columns still exist after df_new dropping
missing = [v for v in COL.values() if v not in df_new.columns]
if missing:
    raise ValueError(f"After dropping columns, these required columns are missing from df_new: {missing}")

In [17]:
def build_hazards(row):
    data = {}
    for hz in HAZARDS:
        score_col  = f"{hz} - Hazard Type Risk Index Score"
        rating_col = f"{hz} - Hazard Type Risk Index Rating"
        score  = row.get(score_col)  if score_col  in row.index else None
        rating = row.get(rating_col) if rating_col in row.index else None
        if pd.notna(score) or pd.notna(rating):
            entry = {}
            if pd.notna(score):
                try: entry["score"] = float(score)
                except: pass
            if pd.notna(rating):
                entry["rating"] = str(rating)
            if entry:
                data[hz] = entry
    return json.dumps(data, ensure_ascii=False)

hazards_json = df_new.apply(build_hazards, axis=1)

In [18]:
out = pd.DataFrame({
    "county_fips": county_fips,
    "county_name": df_new[COL["county_name"]],
    "state_name" : df_new[COL["state_name"]],
    "state_fips" : state_fips_norm,
    "risk_index_score" : pd.to_numeric(df_new[COL["risk_score"]], errors="coerce"),
    "risk_index_rating": df_new[COL["risk_rating"]],
    "hazards": hazards_json
})

In [19]:
# Basic cleanup & validations
out = out[out["county_fips"].str.fullmatch(r"\d{5}") == True]
print("Rows (before dedupe):", len(out))
print("Unique county_fips   :", out["county_fips"].nunique())

out = out.drop_duplicates(subset=["county_fips"])
print("Rows (after dedupe)  :", len(out))

# Spot-check for well-known FIPS from your sample:
for sample in ["01001","06037","02282","11001","12086"]:
    print(sample, "present? ->", sample in set(out["county_fips"]))

Rows (before dedupe): 3231
Unique county_fips   : 3231
Rows (after dedupe)  : 3231
Expected ~3,200–3,300 rows (depends on release/territories).
01001 present? -> True
06037 present? -> True
02282 present? -> True
11001 present? -> True
12086 present? -> True


In [20]:
out.to_csv(OUT_PATH, index=False)
print("\nSaved:", OUT_PATH)
print("Columns:", list(out.columns))


Saved: /kaggle/working/nri_counties_clean.csv
Columns: ['county_fips', 'county_name', 'state_name', 'state_fips', 'risk_index_score', 'risk_index_rating', 'hazards']


In [22]:
out.head()

,county_fips,county_name,state_name,state_fips,risk_index_score,risk_index_rating,hazards
0,01001,Autauga,Alabama,01,49.220490,Relatively Low,"{""Avalanche"": {""rating"": ""Not Applicable""}, ""Coastal Flooding"": {""rating"": ""Not Applicable""}, ""Cold Wave"": {""score"": 0.0, ""rating"": ""No Rating""}, ""Drought"": {""score"": 50.65224307986, ""rating"": ""Relatively Low""}, ""Earthquake"": {""score"": 57.0792236716513, ""rating"": ""Very Low""}, ""Hail"": {""score"": 57.1110404072542, ""rating"": ""Relatively Low""}, ""Heat Wave"": {""score"": 87.0824053452116, ""rating"": ""Relatively Moderate""}, ""Hurricane"": {""score"": 67.6536563481382, ""rating"": ""Relatively Low""}, ""Ice Storm"": {""score"": 23.8507661558961, ""rating"": ""Very Low""}, ""Landslide"": {""score"": 41.7603597815612, ""rating"": ""Relatively Low""}, ""Lightning"": {""score"": 54.2471042471042, ""rating"": ""Relatively Low""}, ""Riverine Flooding"": {""score"": 52.2430798600064, ""rating"": ""Relatively Low""}, ""Strong Wind"": {""score"": 43.9707286032453, ""rating"": ""Relatively Low""}, ""Tornado"": {""score"": 73.8466433343939, ""rating"": ""Relatively Moderate""}, ""Tsunami"": {""rating"": ""Not Applicable""}, ""Volcanic Activity"": {""rating"": ""Not Applicable""}, ""Wildfire"": {""score"": 29.6213808463252, ""rating"": ""Very Low""}, ""Winter Weather"": {""score"": 12.217626471524, ""rating"": ""Very Low""}}"
1,01003,Baldwin,Alabama,01,97.709195,Relatively High,"{""Avalanche"": {""rating"": ""Not Applicable""}, ""Coastal Flooding"": {""score"": 55.9356136820926, ""rating"": ""Relatively Low""}, ""Cold Wave"": {""score"": 0.0, ""rating"": ""No Rating""}, ""Drought"": {""score"": 86.7960547247852, ""rating"": ""Relatively Moderate""}, ""Earthquake"": {""score"": 69.1695832007636, ""rating"": ""Relatively Low""}, ""Hail"": {""score"": 52.9430480432708, ""rating"": ""Relatively Low""}, ""Heat Wave"": {""score"": 91.0913140311804, ""rating"": ""Relatively Moderate""}, ""Hurricane"": {""score"": 98.8784208165096, ""rating"": ""Relatively High""}, ""Ice Storm"": {""score"": 32.6449033977348, ""rating"": ""Relatively Low""}, ""Landslide"": {""score"": 72.5024092515259, ""rating"": ""Relatively Low""}, ""Lightning"": {""score"": 99.3886743886744, ""rating"": ""Very High""}, ""Riverine Flooding"": {""score"": 87.9414572064906, ""rating"": ""Relatively Moderate""}, ""Strong Wind"": {""score"": 53.7702831689469, ""rating"": ""Relatively Low""}, ""Tornado"": {""score"": 91.3776646516068, ""rating"": ""Relatively Moderate""}, ""Tsunami"": {""rating"": ""Insufficient Data""}, ""Volcanic Activity"": {""rating"": ""Not Applicable""}, ""Wildfire"": {""score"": 92.6503340757238, ""rating"": ""Relatively Moderate""}, ""Winter Weather"": {""score"": 52.0839961819917, ""rating"": ""Relatively Low""}}"
2,01005,Barbour,Alabama,01,56.188355,Relatively Low,"{""Avalanche"": {""rating"": ""Not Applicable""}, ""Coastal Flooding"": {""rating"": ""Not Applicable""}, ""Cold Wave"": {""score"": 0.0, ""rating"": ""No Rating""}, ""Drought"": {""score"": 80.6872414890232, ""rating"": ""Relatively Low""}, ""Earthquake"": {""score"": 44.0025453388482, ""rating"": ""Very Low""}, ""Hail"": {""score"": 39.9936366528794, ""rating"": ""Very Low""}, ""Heat Wave"": {""score"": 69.7104677060134, ""rating"": ""Relatively Low""}, ""Hurricane"": {""score"": 77.1646478241364, ""rating"": ""Relatively Low""}, ""Ice Storm"": {""score"": 18.8874083944037, ""rating"": ""Very Low""}, ""Landslide"": {""score"": 70.7998715065853, ""rating"": ""Relatively Low""}, ""Lightning"": {""score"": 82.3037323037323, ""rating"": ""Relatively Moderate""}, ""Riverine Flooding"": {""score"": 13.967546929685, ""rating"": ""Very Low""}, ""Strong Wind"": {""score"": 35.5074769328667, ""rating"": ""Relatively Low""}, ""Tornado"": {""score"": 73.9420935412027, ""rating"": ""Relatively Moderate""}, ""Tsunami"": {""rating"": ""Not Applicable""}, ""Volcanic Activity"": {""rating"": ""Not Applicable""}, ""Wildfire"": {""